In [17]:
import pickle
from dataclasses import fields
from pathlib import Path

import igl
import matplotlib.pyplot as plt
import numpy as np
import pyvista as pv
import scipy as sp
import seaborn as sns

from cardiac_electrophysiology.ulac import preprocessing

In [18]:
patient_id = "01"
infile_name = f"../data/processed/patient_{patient_id}/mesh_with_fibers_tags.vtk"
load_file = Path("parameterization.pkl")

In [19]:
vtk_mesh = pv.read(infile_name)
triangular_mesh = preprocessing.convert_unstructured_to_polydata_mesh(vtk_mesh)
vertices = np.array(triangular_mesh.points)
simplices = preprocessing.convert_pv_cells_to_numpy_cells(triangular_mesh.faces)

with open(load_file, "rb") as f:
    uac_pv_boundaries, uac_laa_boundary, uac_mv_boundary = pickle.load(f)

In [20]:
boundary_indices = []
boundary_values = []

boundary_indices.append(uac_mv_boundary.inds)
boundary_values.append(np.vstack([uac_mv_boundary.alpha, uac_mv_boundary.beta]).T)
boundary_ind_array = np.concatenate(boundary_indices)
boundary_value_array = np.vstack(boundary_values)

harmonic_mapping = igl.harmonic(vertices, simplices, boundary_ind_array, boundary_value_array, 1)

In [24]:
mapping_z_direction = np.zeros(vertices.shape[0])
mapping = np.hstack([harmonic_mapping, mapping_z_direction[:, None]])

uac_mesh = pv.PolyData.from_regular_faces(mapping, simplices)
plotter = pv.Plotter(window_size=[800, 800])
plotter.add_mesh(
    uac_mesh,
    style="wireframe",
    color="lightgrey",
)
for boundary_field in fields(uac_pv_boundaries):
    if "inner" in boundary_field.name:
        continue
    boundary = getattr(uac_pv_boundaries, boundary_field.name)
    boundary_points = mapping[boundary.inds]
    plotter.add_points(boundary_points, color="red", point_size=10, render_points_as_spheres=True)
laa_boundary_points = mapping[uac_laa_boundary.inds]
plotter.add_points(laa_boundary_points, color="blue", point_size=10, render_points_as_spheres=True)
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:46389/index.html?ui=P_0x7f1790567250_14&reconnect=auto" class="pyv…

In [15]:
smoothend_mesh = uac_mesh.smooth(n_iter=100, relaxation_factor=0.1)

plotter = pv.Plotter(window_size=[800, 800])
plotter.add_mesh(
    smoothend_mesh,
    style="wireframe",
    color="lightgrey",
)
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:46389/index.html?ui=P_0x7f176bb1d450_9&reconnect=auto" class="pyvi…